In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from IPython.display import HTML, display

from nexum.core.models import Perceptron
from nexum.core.layers import OutputLayer
from nexum.services.utils import (
    get_clustered_space_image,
    get_scatters_clustered,
    get_clustered_space_splitter,
)

init_notebook_mode(connected=True)

In [ ]:
def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4></td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

def display_compare_table(input_data, targets, nn):
    yes = "✅"
    no = "❌"
    row_table = [["data", "target", "result", "assert"], ]
    
    for i in range(input_data.shape[0]):
        result = nn.predict(input_data[i])
        assert_result = yes if np.array_equal(targets[i], result) else no
        row_table.append([input_data[i], targets[i], result, assert_result])
        
    display_table(row_table)
    
step_function = np.vectorize(lambda x: int(x >= 0.5))

## Завдання 1

In [ ]:
o_l = OutputLayer(1)
nn = Perceptron(layers_config=[2, o_l])
nn.finalize = step_function

input_data = np.array(
    [
        [1, 1],
        [1, -1],
        [1.4, 1.5],
        [2, 0.5],
        [2, -2],
    ]
)

targets = np.array(
    [
        [1],
        [1],
        [1],
        [0],
        [0],
    ]
)

nn.train(input_data, targets, epochs=50, learning_rate=0.1)

display_compare_table(input_data, targets, nn)
image = get_clustered_space_image(nn, input_data)
traces = get_scatters_clustered(nn, input_data)
split_traces = get_clustered_space_splitter(nn, input_data)

layout = go.Layout()
fig = go.Figure([image, *traces, *split_traces], layout=layout)
fig.update_layout(
    yaxis = dict(autorange="reversed")
)
iplot(fig)

## Завдання 2

In [ ]:
# o_l = OutputLayer(1, activation_function="relu")
o_l = OutputLayer(1)
nn = Perceptron(layers_config=[3, o_l])
nn.finalize = step_function

input_data = np.array(
    [
        (-1.5, -1.5, -1.5),
        (1.5, -1.5, -1.5),
        (-1.5, 1.5, -1.5),
        (1.5, 1.5, -1.5),
        (-1.5, -1.5, 1.5),
        (1.5, -1.5, 1.5),
        (-1.5, 1.5, 1.5),
        (1.5, 1.5, 1.5),
    ]
)

targets = np.array(
    [
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
    ]
)

nn.train(input_data, targets, epochs=50, learning_rate=0.1)

traces = get_scatters_clustered(nn,input_data)
split_traces = get_clustered_space_splitter(nn, input_data)

layout = go.Layout()
fig = go.Figure([*traces, *split_traces], layout=layout)
iplot(fig)

## Завдання 3

In [ ]:
# o_l = OutputLayer(2, activation_function="relu")
o_l = OutputLayer(2)
nn = Perceptron(layers_config=[2, o_l])
nn.finalize = step_function

input_data = np.array(
    [
        (0.5, 1.3),
        (0.8, 1.6),
        (0.9, 1.8),
        (1.2, 0.6),
        (1.5, 0.8),
        (0.2, 0.1),
        (0.1, 0.5),
        (-0.2, 0.6),
        (-0.6, -0.8),
        (-1.0, -1.2),
    ]
)

targets = np.array(
    [
        (0, 1),
        (0, 1),
        (0, 1),
        (1, 1),
        (1, 1),
        (0, 0),
        (0, 0),
        (0, 0),
        (1, 0),
        (1, 0),
    ]
)

nn.train(input_data, targets, epochs=10000, learning_rate=0.05)

display_compare_table(input_data, targets, nn)

image = get_clustered_space_image(nn, input_data)
traces = get_scatters_clustered(nn, input_data)
split_traces = get_clustered_space_splitter(nn, input_data)

layout = go.Layout()
fig = go.Figure([image, *traces, *split_traces], layout=layout)
iplot(fig)

## Extra

In [ ]:
from sklearn.datasets import load_iris 

def one_hot(indices):
    unique_values = list(np.unique(indices))
    depth = len(unique_values)
    results = np.zeros((indices.shape[0], depth))
    
    for i in range(indices.shape[0]):
        index = unique_values.index(indices[i])
        results[i][index] = 1
        
    return results

def get_training_data(x, y, training_size: float):
    # x = x.copy()
    randomize = np.arange(len(x))
    np.random.shuffle(randomize)
    
    training_x = x[randomize]
    training_y = y[randomize]
    
    training_n = int(training_size * len(x))
    
    return training_x[:training_n+1], training_y[:training_n+1]
    

iris = load_iris()
 
training_size = 0.3
epochs = 1000
learning_rate = 0.08

X = iris['data'] 
y = iris['target'] 
names = iris['target_names'] 
feature_names = iris['feature_names']
y = one_hot(y)

t_x, t_y, = get_training_data(X, y, training_size)

o_l = OutputLayer(3, activation_function="softmax")
nn = Perceptron(layers_config=[4, 5, o_l])
nn.finalize = step_function

nn.train(t_x, t_y, epochs=epochs, learning_rate=learning_rate)

display_compare_table(X, y, nn)